In [1]:
import pdal
import smrf

import numpy as np
import pandas as pd
import rasterio

Read in some data sourced from a YellowScan, courtesy of the StREAM Lab at Virginia Tech.

This record is approximately 54 million points.

In [2]:
fn_in = 'data/StREAM_20170405.laz'

# Commands can be chaned together with the pipe:
pipeline = pdal.Reader(fn_in).pipeline()
pipeline.execute()

arr = pipeline.arrays[0]

Apply SRMF to remove the vegetation.  Although the point density of scan is quite high (approximately 430 points per square meter), it isn't necessary that SMRF's cellsize corresponds to this value.  A 1 meter grid will often work just as well, and run significantly faster.  A DTM can be generated from the classified output in the next step at whatever resolution is appropriate, and need not be the same as is used here.

Because this area has very few large buildings and is dominated by relatively small vegetation, a very small window can be used.  Even large trees don't require a radius setting here that's very large, because the lidar will typically penetrate through the canopy, and the minimum return for that grid cell will often still be ground.

AGH is above ground height, which can be used to classify vegetation.  We'll use some easy breakpoints to define low, medium, and high vegetation for this, but there are likely much more sophisticated ways to do this.

In [3]:
Z,T,obj_cells,obj_array,extras = smrf.smrf(arr['X'],arr['Y'],arr['Z'],cellsize=1,windows=2,slope_threshold=.15,
                                            elevation_threshold=.1,elevation_scaler=0,return_extras=True)

AGH = extras['above_ground_height']

In [4]:
with rasterio.open('out/StREAM_20170405.dtm.tif', 'w', driver='GTiff', 
                             height=Z.shape[0], width=Z.shape[1], 
                             count=1, dtype=np.float32, transform=T) as src:
    src.write(Z.astype(np.float32), 1)

In [5]:
new_class = 4*np.ones_like(arr['Classification'])  # Assume everything else is medium vegetation
new_class[~obj_array] = 2                          # Ground
new_class[(obj_array) & (AGH<1)] = 3               # Unless under a meter, then it's low vegetation
new_class[(obj_array) & (AGH>3)] = 5               # Or higher than 3 meters, then it's high vegetation


arr['Classification'] = new_class

pipeline2 = pdal.Writer.las(
    filename="out/StREAM_20170405.classified.laz",
    forward="all",
).pipeline(arr)

pipeline2.execute()

54355116